In [10]:
import os
import zipfile
import pandas as pd

DOWNLOAD_DIR = os.path.join(
    os.getcwd(), 'election-results-scraper/scraped_files')
SOURCE_DIR = os.path.join(os.getcwd(), 'data/source/')
COUNTY_RESULTS_DIR = os.path.join(SOURCE_DIR, 'county_election_results')
PRECINCT_RESULTS_DIR = os.path.join(SOURCE_DIR, 'precinct_election_results')
STATE_RESULTS_DIR = os.path.join(SOURCE_DIR, 'state_election_results')


def unzipCountyResults():
    countyZipFiles = [f for f in os.listdir(DOWNLOAD_DIR) if f.split('_')[
        1].startswith('County')]
    for f in countyZipFiles:
        with zipfile.ZipFile(os.path.join(DOWNLOAD_DIR, f), 'r') as z:
            z.extractall(COUNTY_RESULTS_DIR)


def unzipPrecinctResults():
    precinctZipFiles = [f for f in os.listdir(DOWNLOAD_DIR) if f.split('_')[
        1].startswith('Precinct')]
    for f in precinctZipFiles:
        with zipfile.ZipFile(os.path.join(DOWNLOAD_DIR, f), 'r') as z:
            z.extractall(PRECINCT_RESULTS_DIR)


def moveStateResults():
    stateResultsFiles = [f for f in os.listdir(DOWNLOAD_DIR) if f.split('_')[
        1].startswith('State')]
    print(
        f"Moving {len(stateResultsFiles)} files from ${DOWNLOAD_DIR} to {STATE_RESULTS_DIR}")
    for f in stateResultsFiles:
        os.rename(os.path.join(DOWNLOAD_DIR, f),
                  os.path.join(STATE_RESULTS_DIR, f))


In [28]:
stateResultsFiles = [f for f in os.listdir(STATE_RESULTS_DIR)]
allResults = pd.concat([pd.read_csv(os.path.join(STATE_RESULTS_DIR, f),
                       engine='python', delimiter=",", encoding='utf-8', index_col=False) for f in stateResultsFiles])
allResults.head()

,elec_date,entity_description,race_number,race_description,race_party,tot_race_prec,race_prec_reporting,cand_number,cand_name,cand_party,cand_absmail_votes,cand_early_votes,cand_elecday_votes,cand_tot_votes,race_county_owner
0,4/7/2015,"FEDERAL, STATE AND COUNTY",11001,FOR STATE SENATOR DISTRICT 11 (UNEXPIRED TERM),DEM,35.0,35.0,1,KEVIN MATTHEWS,DEM,88,73,1458,1619,NaN
1,4/7/2015,"FEDERAL, STATE AND COUNTY",11001,FOR STATE SENATOR DISTRICT 11 (UNEXPIRED TERM),DEM,35.0,35.0,2,REGINA GOODWIN,DEM,73,95,1149,1317,NaN
2,4/7/2015,"FEDERAL, STATE AND COUNTY",11001,FOR STATE SENATOR DISTRICT 11 (UNEXPIRED TERM),DEM,35.0,35.0,3,HEATHER NASH,DEM,17,12,457,486,NaN
3,4/7/2015,BECKHAM COUNTY,30001,COUNTY QUESTION,NaN,13.0,13.0,1,FOR THE PROPOSITION - YES,NaN,41,34,1272,1347,BECKHAM
4,4/7/2015,BECKHAM COUNTY,30001,COUNTY QUESTION,NaN,13.0,13.0,2,AGAINST THE PROPOSITION - NO,NaN,15,15,919,949,BECKHAM


,elec_date,entity_description,race_number,race_description,race_party,tot_race_prec,race_prec_reporting,cand_number,cand_name,cand_party,cand_absmail_votes,cand_early_votes,cand_elecday_votes,cand_tot_votes,race_county_owner
0,4/7/2015,"FEDERAL, STATE AND COUNTY",11001,FOR STATE SENATOR DISTRICT 11 (UNEXPIRED TERM),DEM,35.0,35.0,1,KEVIN MATTHEWS,DEM,88,73,1458,1619,NaN
1,4/7/2015,"FEDERAL, STATE AND COUNTY",11001,FOR STATE SENATOR DISTRICT 11 (UNEXPIRED TERM),DEM,35.0,35.0,2,REGINA GOODWIN,DEM,73,95,1149,1317,NaN
2,4/7/2015,"FEDERAL, STATE AND COUNTY",11001,FOR STATE SENATOR DISTRICT 11 (UNEXPIRED TERM),DEM,35.0,35.0,3,HEATHER NASH,DEM,17,12,457,486,NaN
3,4/7/2015,BECKHAM COUNTY,30001,COUNTY QUESTION,NaN,13.0,13.0,1,FOR THE PROPOSITION - YES,NaN,41,34,1272,1347,BECKHAM
4,4/7/2015,BECKHAM COUNTY,30001,COUNTY QUESTION,NaN,13.0,13.0,2,AGAINST THE PROPOSITION - NO,NaN,15,15,919,949,BECKHAM
